In [2]:
%pip install scikit-learn
%pip install pandas
%pip install seaborn
%pip install torch
%pip install stg 
%pip install matplotlibv


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement matplotlibv (from versions: none)
ERROR: No matching distribution found for matplotlibv

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached stg-0.1.2-py3-none-any.whl
  Using cached lifelines-0.28.0-py3-none-any.whl.metadata (3.2 kB)
  Using cached autograd-1.6.2-py3-none-any.whl.metadata (706 bytes)
  Using cached autograd_gamma-0.5.0-py3-none-any.whl
  Using cached formulaic-1.0.1-py3-none-any.whl.metadata (6.1 kB)
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB 1.4 MB/s eta 0:00:02
   - -------------------------------------- 0.1/2.7 MB 1.1 MB/s eta 0:00:03
   -- ------------------------------------- 0.2/2.7 MB 1.1 MB/s eta 0:00:03
   --- ------------------------------------ 0.3/2.7 MB 1.4 MB/s eta 0:00:02
   ---- ----------------------------------- 0.3/2.7 MB 1.3 MB/s eta 0:00:02
   ------ --------------------------------- 0.5/2.7 MB 1.9 MB/s eta 0:00:02
   -------- ------------------------------- 0.6/2.7 MB 1.8 MB/s eta 0:00:02
   --------- ------------------------------ 0.6/2.7 MB 1.8 MB/s eta 0:00:02
   --------------- --------


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sns
import torch
from sklearn.model_selection import train_test_split
from stg import STG  # Assuming this is your custom model import
import matplotlib.pyplot as plt
import os


In [4]:
# Load the MADELON dataset package
path = os.getcwd() + "\madelon.csv"
madelon = pd.read_csv(path)

# Print the first few rows of the DataFrame to verify the data
print(madelon.head())

# Split data into features and target
X = madelon.drop('Class', axis=1)
y = madelon['Class']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert DataFrames to numpy arrays
X_train_np = X_train.to_numpy()
X_test_np = X_test.to_numpy()
y_train_np = y_train.to_numpy()
y_test_np = y_test.to_numpy()

    V1   V2   V3   V4   V5   V6   V7   V8   V9  V10  ...  V492  V493  V494  \
0  485  477  537  479  452  471  491  476  475  473  ...   481   477   485   
1  483  458  460  487  587  475  526  479  485  469  ...   478   487   338   
2  487  542  499  468  448  471  442  478  480  477  ...   481   492   650   
3  480  491  510  485  495  472  417  474  502  476  ...   480   474   572   
4  484  502  528  489  466  481  402  478  487  468  ...   479   452   435   

   V495  V496  V497  V498  V499  V500  Class  
0   511   485   481   479   475   496      2  
1   513   486   483   492   510   517      2  
2   506   501   480   489   499   498      2  
3   454   469   475   482   494   461      1  
4   486   508   481   504   495   511      1  

[5 rows x 501 columns]


In [5]:

# Define model parameters
args_cuda = torch.cuda.is_available()
device = torch.device("cuda" if args_cuda else "cpu")
feature_selection = True

# Initialize and fit the model
model = STG(task_type='regression',
            input_dim=X_train_np.shape[1],
            output_dim=1,
            hidden_dims=[500, 50, 10],
            activation='tanh',
            optimizer='SGD',
            learning_rate=0.1,
            batch_size=X_train_np.shape[0],
            feature_selection=feature_selection,
            sigma=0.5,
            lam=0.1,
            random_state=1,
            device=device)

model.fit(X_train_np, y_train_np, nr_epochs=3000, valid_X=X_test_np, valid_y=y_test_np, print_interval=1000)


c:\Users\yonat\Documents\GeomteryOfInformation\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([2080])) that is different to the input size (torch.Size([2080, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\yonat\Documents\GeomteryOfInformation\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([520])) that is different to the input size (torch.Size([520, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 1000: loss=0.333937 valid_loss=0.249973
Epoch: 2000: loss=0.333347 valid_loss=0.249984
Epoch: 3000: loss=0.332924 valid_loss=0.249972


In [6]:
# Evaluate the model
tmp_loss = torch.nn.MSELoss()
train_loss = tmp_loss(torch.Tensor(y_train_np), torch.Tensor(model.predict(X_train_np)))
test_loss = tmp_loss(torch.Tensor(y_test_np), torch.Tensor(model.predict(X_test_np)))

print(f"Train Loss: {train_loss.item()}")
print(f"Test Loss: {test_loss.item()}")


Train Loss: 0.25008729100227356
Test Loss: 0.24997229874134064


c:\Users\yonat\Documents\GeomteryOfInformation\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([2080, 1])) that is different to the input size (torch.Size([2080])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\yonat\Documents\GeomteryOfInformation\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([520, 1])) that is different to the input size (torch.Size([520])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [ ]:
# X_train_np
# X_test_np 
# y_train_np
# y_test_np
# Plotting the results
y_pred_train = model.predict(X_train_np)
y_pred_test = model.predict(X_test_np)

# Calculate confusion matrices for both training and testing sets
y_train_flat = y_train_np.ravel()
y_pred_train_flat = y_pred_train.ravel()
conf_matrix_train = confusion_matrix(y_train_flat, y_pred_train_flat)
# conf_matrix_test = confusion_matrix(y_test_np, y_pred_test)


In [ ]:

# Plot confusion matrix for training set
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
sns.heatmap(conf_matrix_train, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix - Training Set')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')

# Plot confusion matrix for testing set
plt.subplot(1, 2, 2)
sns.heatmap(conf_matrix_test, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix - Testing Set')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')

plt.tight_layout()
plt.show()